In [1]:
import requests
import pandas as pd
import json
from shapely.geometry import Point
import geopandas as gpd

# Analyzing Aggravated Burglaries in Davidson County

### Part 1 - Data Gathering using APIs

1.find all aggravated burglary incidents () that were reported during the nine month period from January 1, 2022 through September 30, 2022. (**Hint:** Check out the [API Docs](https://dev.socrata.com/foundry/data.nashville.gov/2u6v-ujjs) to see how to narrow down the response to just the desired results).

**Aggrivated Burglary** - TIBRS Data Collection Manual states that TN uses NIBRS for coding offenses under TN Code Annotated 39-14-403.  The following NIBRS codes are used for aggrivated burglary in Tennessee
- Burglary - 220
- Assualt - 13A
- Robery - 120
- Weapon Law Violation - 520

In [2]:
#MPD dataset API
# Load API Key into dictionary: credentials
with open('app_token.json') as fi:
    credentials = json.load(fi)
# Returns value associated with dictionary key
api_key = credentials['token']
#List of dates of interest
dates = ("'2022-01-01T00:00:00'", "'2022-09-30T23:59:59'")
#NIBRS codes for Aggrivated burglary
nibrs = ['220', '13A', '120', '520']

In [3]:
# API Calls for Nashville's MPD dataset
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'
# Parameter dictionary for API call
params = {
    # Find information between Dates
    '$where': f'incident_occurred between '+dates[0]+' and '+dates[1],
    # Find NIBRS code 220: Burglary 
    'offense_description': 'BURGLARY- AGGRAVATED',
    # Limit changed to allow more results than default
    '$limit': '10000',
    # Provides api key for query
    '$$app_token': api_key
}
# Stores API response as a requests object: response
response = requests.get(endpoint)
# Applies the parameters dictionary to API response
response = requests.get(endpoint, params = params)

In [4]:
# Checks response code.  We want to see "200"
response

<Response [200]>

In [5]:
#formats response to .json()
res = response.json()
burglaries = pd.DataFrame(res)
burglaries["latitude"] = pd.to_numeric(burglaries["latitude"])
burglaries["longitude"] = pd.to_numeric(burglaries["longitude"])
burglaries["geometry"] = burglaries.apply(lambda x: Point((x['longitude'], x['latitude'])), axis=1)

/home/chunderdamus/anaconda3/envs/geospatial_ds6/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


2. Using the [2020 American Community Survey API](https://www.census.gov/data/developers/data-sets/acs-5year.html), obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. 

In [6]:
#2020 American Community Survey API Calls
# Load API Key into dictionary: credentials
with open('census_key.json') as fi:
    credentials = json.load(fi)
# Returns value associated with dictionary key
census_api = credentials['api_key']
#FPIS codes for Tennessee, davidson county
fpis= ['47', '37']
#Datasets of interest
Census_Datasets = ('NAME','B01001_001E','S1901_C01_012E')

In [7]:
#2020 American Community Survey for population
#Gave up trying to systematically pass the variables.
endpoint = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S0101_C01_001E&for=tract:*&in=state:47&in=county:037'
# Parameter dictionary for API call
params = {
    'key': census_api
}
# Stores API response as a requests object: response
response = requests.get(endpoint)
# Applies the parameters dictionary to API response
response = requests.get(endpoint, params = params)

In [8]:
# Checks response code.  We want to see "200"
response.text

'[["NAME","S0101_C01_001E","state","county","tract"],\n["Census Tract 190.04, Davidson County, Tennessee","4470","47","037","019004"],\n["Census Tract 190.07, Davidson County, Tennessee","2904","47","037","019007"],\n["Census Tract 190.08, Davidson County, Tennessee","6005","47","037","019008"],\n["Census Tract 191.05, Davidson County, Tennessee","5707","47","037","019105"],\n["Census Tract 191.06, Davidson County, Tennessee","4336","47","037","019106"],\n["Census Tract 191.08, Davidson County, Tennessee","3478","47","037","019108"],\n["Census Tract 191.09, Davidson County, Tennessee","5347","47","037","019109"],\n["Census Tract 191.10, Davidson County, Tennessee","3604","47","037","019110"],\n["Census Tract 191.11, Davidson County, Tennessee","4715","47","037","019111"],\n["Census Tract 191.12, Davidson County, Tennessee","5013","47","037","019112"],\n["Census Tract 191.15, Davidson County, Tennessee","3661","47","037","019115"],\n["Census Tract 191.16, Davidson County, Tennessee","73

In [9]:
#formats response to .json()
res = response.json()
# creates pandas dataframe from res variable (response.json)
population_df = pd.DataFrame(res)
header = population_df.iloc[0]
population_df = population_df[1:]
population_df.columns = header
population_df = population_df.rename(columns={'S0101_C01_001E': 'population'})
population_df

,NAME,population,state,county,tract
1,"Census Tract 190.04, Davidson County, Tennessee",4470,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",2904,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",6005,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",5707,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",4336,47,037,019106
...,...,...,...,...,...
170,"Census Tract 189.01, Davidson County, Tennessee",3280,47,037,018901
171,"Census Tract 189.02, Davidson County, Tennessee",2704,47,037,018902
172,"Census Tract 189.04, Davidson County, Tennessee",3410,47,037,018904
173,"Census Tract 189.05, Davidson County, Tennessee",3292,47,037,018905


In [10]:
#2020 American Community Survey for Median household income
endpoint = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S1901_C01_012E&for=tract:*&in=state:47&in=county:037'
# Parameter dictionary for API call
params = {
    'key': census_api
}
# Stores API response as a requests object: response
response = requests.get(endpoint)
# Applies the parameters dictionary to API response
response = requests.get(endpoint, params = params)

In [11]:
response.text

'[["NAME","S1901_C01_012E","state","county","tract"],\n["Census Tract 190.04, Davidson County, Tennessee","44001","47","037","019004"],\n["Census Tract 190.07, Davidson County, Tennessee","48485","47","037","019007"],\n["Census Tract 190.08, Davidson County, Tennessee","50512","47","037","019008"],\n["Census Tract 191.05, Davidson County, Tennessee","48271","47","037","019105"],\n["Census Tract 191.06, Davidson County, Tennessee","65323","47","037","019106"],\n["Census Tract 191.08, Davidson County, Tennessee","34893","47","037","019108"],\n["Census Tract 191.09, Davidson County, Tennessee","57039","47","037","019109"],\n["Census Tract 191.10, Davidson County, Tennessee","55553","47","037","019110"],\n["Census Tract 191.11, Davidson County, Tennessee","47700","47","037","019111"],\n["Census Tract 191.12, Davidson County, Tennessee","69821","47","037","019112"],\n["Census Tract 191.15, Davidson County, Tennessee","121094","47","037","019115"],\n["Census Tract 191.16, Davidson County, Te

In [12]:
#formats response to .json()
res = response.json()
# creates pandas dataframe from res variable (response.json)
Med_Household_income_df = pd.DataFrame(res)
header = Med_Household_income_df.iloc[0]
Med_Household_income_df = Med_Household_income_df[1:]
Med_Household_income_df.columns = header
Med_Household_income_df.rename(columns={'S1901_C01_012E': 'med_house_income'}, inplace = True)
Med_Household_income_df

,NAME,med_house_income,state,county,tract
1,"Census Tract 190.04, Davidson County, Tennessee",44001,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",48485,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",50512,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",48271,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",65323,47,037,019106
...,...,...,...,...,...
170,"Census Tract 189.01, Davidson County, Tennessee",47500,47,037,018901
171,"Census Tract 189.02, Davidson County, Tennessee",75286,47,037,018902
172,"Census Tract 189.04, Davidson County, Tennessee",61750,47,037,018904
173,"Census Tract 189.05, Davidson County, Tennessee",65960,47,037,018905


In [26]:
censusmerge_df = pd.merge(population_df,Med_Household_income_df[['NAME','med_house_income']],on='NAME', how='outer')
censusmerge_df[["population", "med_house_income"]] = censusmerge_df[["population", "med_house_income"]].apply(pd.to_numeric)
censusmerge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NAME              174 non-null    object
 1   population        174 non-null    int64 
 2   state             174 non-null    object
 3   county            174 non-null    object
 4   tract             174 non-null    object
 5   med_house_income  174 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 9.5+ KB


### Part 2 - Spatial Joining and Data Merging

3. Download the 2020 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. 

In [15]:
census_tracts = gpd.read_file('../Notebooks/data/tl_2020_47_tract.shp')
burglaries = gpd.GeoDataFrame(burglaries, geometry=burglaries["geometry"], crs=census_tracts.crs)
burglaries

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location,rpa,zone,zip_code,geometry
0,20220167824_11,20220167824,D,DISPATCHED,O,OPEN,Open,2022-04-07T15:00:00.000,2022-04-08T11:52:00.000,JACKSON ST,...,INDIVIDUAL (18 AND OVER),U,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.8, 36.17]}",NaN,NaN,NaN,POINT (-86.80000 36.17000)
1,20220126184_31,20220126184,D,DISPATCHED,O,OPEN,Open,2022-03-18T02:30:00.000,2022-03-18T06:51:00.000,BENTON AVE,...,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.13]}",8029,817,NaN,POINT (-86.77000 36.13000)
2,20220027854_12,20220027854,D,DISPATCHED,O,OPEN,Open,2022-01-18T07:45:00.000,2022-01-19T23:48:00.000,CANE RIDGE RD,...,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.66, 36.04]}",NaN,NaN,NaN,POINT (-86.66000 36.04000)
3,20220032825_11,20220032825,D,DISPATCHED,O,OPEN,Open,2022-01-23T00:40:00.000,2022-01-24T06:57:00.000,BROOKWOOD TER,...,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.86, 36.13]}",5019,121,NaN,POINT (-86.86000 36.13000)
4,20220023736_11,20220023736,D,DISPATCHED,O,OPEN,Open,2022-01-15T10:00:00.000,2022-01-17T11:25:00.000,WEDGEWOOD AVE,...,BUSINESS,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-86.77, 36.13]}",8025,817,NaN,POINT (-86.77000 36.13000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,20220462314_11,20220462314,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,2022-09-13T17:00:00.000,2022-09-15T02:04:00.000,1524 1524,...,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.744, 36....",NaN,NaN,37206,POINT (-86.74400 36.19400)
1338,20220397543_11,20220397543,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,2022-08-11T18:00:00.000,2022-08-11T22:47:00.000,930 930,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.835, 36....",NaN,NaN,37209,POINT (-86.83500 36.16000)
1339,20220487005_11,20220487005,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,2022-09-27T19:16:00.000,2022-09-27T22:46:00.000,400 400,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.794, 36....",NaN,NaN,37207,POINT (-86.79400 36.21400)
1340,20220116140_31,20220116140,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2022-03-13T01:49:00.000,2022-03-13T01:49:00.000,306 306,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.757, 36....",NaN,NaN,37206,POINT (-86.75700 36.18000)


4. Aggregate the data by census tract. **Warning:** each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents. Which census tract had the highest number of burglaries? Which census tract had the highest number of burglaries per 1000 residents? **Note:** Make sure that you keep all census tracts, not just those that have had a burglary.

5. Merge in the census data that you gathered in question 2. Remove any rows that have zero population or negative median income values.

### Part 3 - Statistical Modeling

6. Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. Start with some EDA to look at the relationship between median income and number of aggravated burglaries.

7. Fit a Poisson regression model with target variable the rate of burglaries per census tract and with predictor the median income. Offset using the log of the population so that we are looking at the rate of burglaries per population instead of the number of burglaries. How can you interpret the meaning of the output?

8. **Bonus:** Try out a negative binomial model. To get started with a negative binomial model, you can check out [this tutorial](https://timeseriesreasoning.com/contents/negative-binomial-regression-model/). How does this model compare to the Poisson model?

Additional Resources for Generalized Linear Models:
* DataCamp - [Generalized Linear Models in Python](https://learn.datacamp.com/courses/generalized-linear-models-in-python)
* [Beyond Multiple Linear Regression, Chapter 4](https://bookdown.org/roback/bookdown-BeyondMLR/ch-poissonreg.html) Warning - the code in this book is all R, but the conceptual explanations are very clear.
* [This set of notes](https://apwheele.github.io/MathPosts/PoissonReg.html#negative-binomial-when-the-poisson-does-not-fit), which talks about the problem of overdispersion.